In [7]:
import pandas as pd

In [8]:
df=pd.read_csv("/content/sample_data/train.csv")

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [11]:
x=df.drop("label",axis=1)

In [12]:
y=df["label"]

In [13]:
x.shape

(20800, 4)

In [14]:
y.shape

(20800,)

In [15]:
import tensorflow as tf

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import  pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [17]:
voc_size=5000

In [18]:
messages=x.copy()

In [19]:
messages["title"][101]

'American Drivers Regain Appetite for Gas Guzzlers - The New York Times'

In [20]:
messages.reset_index(inplace=True)

In [21]:
messages["title"][101]

'American Drivers Regain Appetite for Gas Guzzlers - The New York Times'

In [22]:
import nltk
from nltk.corpus import stopwords
import re

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
#Text Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
 
  review = re.sub('[^a-zA-Z]', ' ',str(messages["title"][i]))
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [28]:
corpus[5]

'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'

In [29]:
onehot_encode=[one_hot(words,voc_size) for words in corpus]
onehot_encode[10]


[138, 3643, 794, 4529, 777, 3761, 1339, 3921, 1678, 4515]

In [30]:
#Embedding
sen_length=20
embedded_doc=pad_sequences(onehot_encode,padding='pre',maxlen=sen_length)
print(embedded_doc[25])

[   0    0    0    0    0    0    0    0    0    0    0    0    0 1423
  731 4117 2586 3534  593 1904]


In [31]:
#building a model

embedding_vec_features=40    #40 dimensionality
model=Sequential()
model.add(Embedding(voc_size,embedding_vec_features,input_length=sen_length))
model.add(Dropout(0.3))
model.add(LSTM(100))   
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid')) #output layer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
import numpy as np
X_final=np.array(embedded_doc) 
Y_final=np.array(y)
X_final.shape,Y_final.shape


((20800, 20), (20800,))

In [33]:
#Split the dataset for train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,Y_final,test_size=0.3,random_state=42)


In [46]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
y_test.shape

(14560, 20)
(6240, 20)
(14560,)


(6240,)

In [34]:
#model fitting
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
228/228 [==============================] - 11s 37ms/step - loss: 0.3170 - accuracy: 0.8576 - val_loss: 0.1863 - val_accuracy: 0.9261
Epoch 2/10
228/228 [==============================] - 8s 34ms/step - loss: 0.1339 - accuracy: 0.9483 - val_loss: 0.1842 - val_accuracy: 0.9244
Epoch 3/10
228/228 [==============================] - 8s 34ms/step - loss: 0.1031 - accuracy: 0.9615 - val_loss: 0.1970 - val_accuracy: 0.9256
Epoch 4/10
228/228 [==============================] - 8s 34ms/step - loss: 0.0832 - accuracy: 0.9706 - val_loss: 0.2297 - val_accuracy: 0.9279
Epoch 5/10
228/228 [==============================] - 8s 34ms/step - loss: 0.0655 - accuracy: 0.9782 - val_loss: 0.2561 - val_accuracy: 0.9115
Epoch 6/10
228/228 [==============================] - 8s 34ms/step - loss: 0.0499 - accuracy: 0.9826 - val_loss: 0.2707 - val_accuracy: 0.9155
Epoch 7/10
228/228 [==============================] - 8s 34ms/step - loss: 0.0394 - accuracy: 0.9867 - val_loss: 0.2808 - val_accuracy: 0.926

Performance Metrics


In [35]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score

In [36]:
scores=model.evaluate(X_train,y_train,verbose=1,batch_size=200)

73/73 [==============================] - 2s 26ms/step - loss: 0.0100 - accuracy: 0.9975


In [51]:
#y_pred=model.predict_classes(X_test)
y_pred=(model.predict(X_test) > 0.5).astype("int32")
#y_pred=model.predict(X_test)
#classes_y=np.argmax(y_pred,axis=0)

In [52]:
y_pred


array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [53]:
confusion_matrix(y_test,y_pred)

array([[2865,  283],
       [ 183, 2909]])

In [54]:
accuracy_score(y_test,y_pred)

0.9253205128205129